In [3]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document


In [3]:
from langchain.vectorstores import Pinecone as LangChainPinecone


In [4]:
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_27RFJe_URz8CQ48XctGgaJToz5zu3AULqo2PazExsj4fed28UDsqPTgZgAzvWgWVa7eRXR")
index = pc.Index("newindex")



In [5]:
business_docs = [
    "Our support team is available around the clock via chat, email, or phone.",
    "We accept multiple payment options including UPI, debit cards, and corporate invoicing.",
    "Standard delivery takes 3-5 working days, while express delivery is available at an extra charge.",
    "Items can be exchanged or returned within 15 days, provided they are in original condition.",
    "Subscription services renew automatically unless cancelled at least 48 hours before the billing date.",
    "Bulk orders may qualify for special pricing and extended delivery timelines.",
    "Digital products are delivered instantly to the registered email address after payment confirmation.",
    "For warranty-related issues, customers must contact us within the first 12 months of purchase.",
]


In [6]:
embedding=OpenAIEmbeddings()
docs = [Document(page_content=txt) for txt in business_docs]
docs

C:\Users\bhard\AppData\Local\Temp\ipykernel_8644\1279769366.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings()


[Document(metadata={}, page_content='Our support team is available around the clock via chat, email, or phone.'),
 Document(metadata={}, page_content='We accept multiple payment options including UPI, debit cards, and corporate invoicing.'),
 Document(metadata={}, page_content='Standard delivery takes 3-5 working days, while express delivery is available at an extra charge.'),
 Document(metadata={}, page_content='Items can be exchanged or returned within 15 days, provided they are in original condition.'),
 Document(metadata={}, page_content='Subscription services renew automatically unless cancelled at least 48 hours before the billing date.'),
 Document(metadata={}, page_content='Bulk orders may qualify for special pricing and extended delivery timelines.'),
 Document(metadata={}, page_content='Digital products are delivered instantly to the registered email address after payment confirmation.'),
 Document(metadata={}, page_content='For warranty-related issues, customers must contact

In [7]:
#split text in chunks for better efficiency of context vctr size of llm
text_splt=CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
doc_splt=text_splt.split_documents(docs)

In [8]:
from langchain_pinecone import PineconeVectorStore


In [10]:
vectorstore = PineconeVectorStore.from_documents(
    documents=doc_splt,
    embedding=embedding,
    index_name="newindex",
    pinecone_api_key="pcsk_27RFJe_URz8CQ48XctGgaJToz5zu3AULqo2PazExsj4fed28UDsqPTgZgAzvWgWVa7eRXR"
)

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [13]:
llm=ChatOpenAI()

C:\Users\bhard\AppData\Local\Temp\ipykernel_8644\1964788120.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI()


In [14]:
answer_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff"
)

In [15]:
print(answer_chain.run("Do you have 24/7 customer support?"))


C:\Users\bhard\AppData\Local\Temp\ipykernel_8644\2051025263.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(answer_chain.run("Do you have 24/7 customer support?"))


Yes, our support team is available around the clock via chat, email, or phone.
